In [3]:
import numpy as np
import torch
from torch import nn
# from imgFE import imgFE
# from gripperFE import gripperFE

/home/nilay/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import rospy
import serial
import struct
import time
import numpy as np
from std_msgs.msg import Float32
import serial.tools.list_ports as ports

class controlBox_pressure:
    def __init__(self):
        # ------------------------ CONSTANTS ------------------------ #
        self.PMAX = 13 # which maps to about 1.4 bars
        self.arduinoPB = None

    def set_communication(self, aportPB, baudrate):
        try:
            self.arduinoPB = serial.Serial( 
                                port=aportPB,
                                baudrate=baudrate,
                                timeout=0.1 
                                )
            self.arduinoPB.isOpen() # try to open port
            print ("Valve is opened!")
        except IOError: # if port is already opened, close it and open it again and print message
            self.arduinoPB.close()
            self.arduinoPB.open()
            print ("Valve was already open, was closed and opened again!")

    ## Send values to the Pressure Box
    def send_values(self, U1, U2, U3, L1, L2, L3, G, PMAX):
        if U1 > PMAX or U2 > PMAX or U3 > PMAX or L1 > PMAX or L2 > PMAX or L3 > PMAX or G > 1:
            print("Pressure out of range")
            return

        packet = np.array([106,U1,U2,U3,L1,L2,L3,G],dtype = np.uint8)
        
        if self.arduinoPB.isOpen():
            for value in packet : 
                s = struct.pack('!{0}B'.format(len(packet)), *packet)
                self.arduinoPB.write(s)
    
    def valve_shutdown(self):
        print("Closing the valve!!")
        self.send_values(0,0,0,0,0,0,0,self.PMAX) # Release the object
        self.arduinoPB.close()        

    def actuate_finger(self, cycles, pressure_value, aportPB, baudrate, PMAX):
        try:
            start_time = time.time()
            for cycle in range(cycles):
                self.set_communication(aportPB, baudrate) # open the port
                self.send_values(pressure_value,0,0,0,0,0,0,PMAX)  # Open the valve V1 and grasp the object
                self.send_values(0,0,0,0,0,0,0,PMAX) # Release the object
                self.arduinoPB.close() # close the valve
    
            end_time = time.time()
            total_time = (end_time - start_time)
            print(f"Total time taken for the experiment is {total_time}")

        except Exception as e:
            print(f"Error encountered while sending value to control Box, closing the port!!")
            self.send_values(0,0,0,0,0,0,0,PMAX) # Release the object
            self.arduinoPB.close()

In [2]:
frequency = 0.1 # 20 hz
baudrate = 115200
cycles = 3
pressure_value = 10
PMAX = 13
port = '/dev/ttyACM1'
g = controlBox_pressure()
g.set_communication(port, baudrate)
g.valve_shutdown()

Valve is opened!
Closing the valve!!
